In [3]:
%pip install -U langchain-community



   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
    --------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -- ------------------------------------- 0.1/2.5 MB 1.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/2.5 MB 1.5 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.5 MB 1.5 MB/s eta 0:00:02
   ------ --------------------------------- 0.4/2.5 MB 1.5 MB/s eta 0:00:02
   --------------- ------------------------ 1.0/2.5 MB 2.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 4.5 MB/s eta 0:00:01
   ---------------------------------------  2.5/2.5 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 5.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/49.3 kB ? eta -:--:--
   ---------------------------------------- 49.3/49.3 kB 2.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain.vectorstores import Milvus
from langchain.llms import Ollama
from pymilvus import connections, utility, Collection, CollectionSchema, FieldSchema, DataType
from sentence_transformers import SentenceTransformer
import random

# Initialize the embedding model
embedding_fn = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Connect to Milvus
def connect_to_milvus(host="localhost", port="19530"):
    connections.connect("default", host=host, port=port)
    print("Connected to Milvus.")

# Ensure the collection exists in Milvus
def get_or_create_collection(collection_name, fields=None, description=""):
    if not utility.has_collection(collection_name):
        if not fields:
            raise ValueError(f"Collection '{collection_name}' does not exist and no schema provided to create it.")
        schema = CollectionSchema(fields=fields, description=description)
        collection = Collection(name=collection_name, schema=schema)
        print(f"Created collection: {collection_name}")
    else:
        collection = Collection(name=collection_name)
    return collection

# Retrieve relevant context from Milvus based on the user query
def retrieve_context(collection_name, query, top_k=15):
    collection = get_or_create_collection(collection_name)
    query_embedding = embedding_fn.encode(query).tolist()
    
    # Search for the top_k most relevant quotes
    results = collection.search(
        data=[query_embedding],
        anns_field="vector",  # Correct field name for embeddings
        param={"metric_type": "L2", "params": {"nprobe": 10}},  # Use L2 for Euclidean distance
        limit=top_k,
        output_fields=["text"]  # Correct field name for the quote text
    )
    
    context = []
    for result in results[0]:
        context.append(result.entity.text)  # Use dot notation to access the text field
    return context



# Insert a new conversation history into Milvus
def insert_to_milvus(collection_name, question, answer, embedding):
    # Define the primary key field
    primary_key = "id"

    # Create or get the collection schema with a primary key
    if not utility.has_collection(collection_name):
        fields = [
            FieldSchema(name=primary_key, dtype=DataType.INT64, is_primary=True),
            FieldSchema(name="question", dtype=DataType.VARCHAR, max_length=1024),
            FieldSchema(name="answer", dtype=DataType.VARCHAR, max_length=1024),
            FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
        ]
        schema = CollectionSchema(fields, description="Chatbot conversation history")
        collection = Collection(name=collection_name, schema=schema)
        print(f"Collection '{collection_name}' created.")
    else:
        collection = get_or_create_collection(collection_name)

    # Generate a unique ID (e.g., using a random integer or time-based approach)
    conversation_id = random.randint(1000000, 9999999)  # Example random ID, can be replaced with a more unique approach

    # Insert the conversation data
    entities = [
        [conversation_id],  # Provide a valid unique ID
        [question],
        [answer],
        [embedding],
    ]
    
    collection.insert(entities)
    print("Conversation added to Milvus.")

# Chatbot logic with RAG
def chatbot():
    # Connect to Milvus
    connect_to_milvus()
    collection_name = "Miri_Regev_quotes"  # Collection storing the quotes

    # Initialize LLM
    llm = Ollama(model="llama3.2")  # Replace with your specific Ollama model

    print("Chatbot with RAG is ready! Type 'exit' to quit.")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        
        # Retrieve context (quotes) from Milvus
        context_quotes = retrieve_context(collection_name, user_input, top_k=3)
        context_text = "\n".join(context_quotes)

        # Format the prompt with context and user query
        prompt = (
            f"The following are quotes by Miri Regev:\n{context_text}\n\n"
            f"Based on the above quotes, answer the following question like you were Miri Regev, with her tone and common words\phrases she said! answer only to the question, do not show your thinking process. ANSWER IN HEBREW!:\n{user_input}"
        )
        
        # Generate response using the LLM
        response = llm(prompt)
        
        # Store the conversation in Milvus
        question_embedding = embedding_fn.encode(user_input).tolist()
        insert_to_milvus("chatbot_history", user_input, response, question_embedding)
        
        # Display the response
        print(f"Bot: {response}")

if __name__ == "__main__":
    chatbot()


In [7]:
from pymilvus import Collection

collection = Collection("Miri_Regev_quotes")
print(collection.schema)


{'auto_id': False, 'description': 'Demo collection for quotes of Miri Regev', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}], 'enable_dynamic_field': False}


In [ ]:
if __name__ == "__main__":
    chatbot()

NameError: name 'chatbot' is not defined